In [ ]:
!pip install fastapi
!pip install uvicorn
!pip install pickle5
!pip install pydantic
!pip install scikit-learn
!pip install requests
!pip install pypi-json
!pip install pyngrok
!pip install nest-asyncio
!pip install shapely
!pip install geopandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.2/58.2 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.1/132.1 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pickle5: filename=pickle5-0.0.11-cp310-cp310-linux_x86_64.whl size=256415 sha256=40636dd01f394c5bfaa247baeede648805480e5dc146820c0ae3af2104f8cdc4
  Stored in directory: /root/.cache/pip/wheels/7d/14/ef/4aab19d27fa8e58772be5c71c16add0426acf9e1f64353235c
Successfully built pickle5
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.3/99.3 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.0/127.0 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━

In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel
import pickle
import json
import uvicorn
from pyngrok import ngrok
from fastapi.middleware.cors import CORSMiddleware
import nest_asyncio
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point
import folium
import shapely.geometry as geometry
from pyproj import Proj, transform

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

#for maha
mhmap1 = gpd.read_file('/content/drive/MyDrive/fyp/Flood shapefiles/2007/Max_flood_EventDec2007.zip')
mhmap2 = gpd.read_file('/content/drive/MyDrive/fyp/Flood shapefiles/2008/Max_flood_EventNovDec2008.zip')
mhmap3 = gpd.read_file('/content/drive/MyDrive/fyp/Flood shapefiles/2014/Max_flood_EventDec2014.zip')
mhmap4 = gpd.read_file('/content/drive/MyDrive/fyp/Flood shapefiles/2021/2021.11.11 - flood.zip')
mhmap5 = gpd.read_file('/content/drive/MyDrive/fyp/Flood shapefiles/2022/2022.12.18 - flood.zip')

#for yala
ylmap1 = gpd.read_file('/content/drive/MyDrive/fyp/Flood shapefiles/2003/2003.05.20 - flood.zip')
ylmap2 = gpd.read_file('/content/drive/MyDrive/fyp/Flood shapefiles/2003/2003.05.22 - flood.zip')
ylmap3 = gpd.read_file('/content/drive/MyDrive/fyp/Flood shapefiles/2003/2003.05.26 - flood.zip')
ylmap4 = gpd.read_file('/content/drive/MyDrive/fyp/Flood shapefiles/2010/2010.05.18 - flood.zip')
ylmap5 = gpd.read_file('/content/drive/MyDrive/fyp/Flood shapefiles/2010/2010.05.21 - flood.zip')
ylmap6 = gpd.read_file('/content/drive/MyDrive/fyp/Flood shapefiles/2010/2010.05.22 - flood.zip')
ylmap7 = gpd.read_file('/content/drive/MyDrive/fyp/Flood shapefiles/2021/2021.06.03 - flood.zip')

#for adm boundry
shapefile_path = "/content/drive/MyDrive/fyp/L4.zip"
provinces = gpd.read_file(shapefile_path)


# Run both flod risk and admin boundry checking

In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel

app = FastAPI()


origins = ["*"]

app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)



class GreetRequest(BaseModel):
    season: str
    lat: float
    lon: float

@app.post("/flood_risk")
def greet(request: GreetRequest):
    season = request.season
    #location = request.location

    #location = (11.228193200490075, 78.23941720427615)
    #season = "maha"
    latitude = request.lat
    longitude = request.lon

    # Define the coordinates in EPSG:4326 (latitude, longitude)
    # latitude = 9.761907869860229
    # longitude = 80.08335575093017

    # Define the EPSG codes for EPSG:4326 (WGS84) and EPSG:5235
    epsg_4326 = Proj(init='epsg:4326')
    epsg_5235 = Proj(init='epsg:5235')

    # Convert the coordinates to EPSG:5235
    x, y = transform(epsg_4326, epsg_5235, longitude, latitude)

    newlat = x
    newlon = y

    # Print the converted coordinates
    # print("-----------------------")
    # print(newlat,newlon)

    # for epsg:4326 point
    normal_point = geometry.Point(longitude,latitude)

    # for epsg:5235 point
    converted_point = geometry.Point(newlat,newlon)

    if season == "maha" or season == "මහ":

        maha_flood_risk_list = {"mhmap1": "", "mhmap2": "", "mhmap3": "", "mhmap4": "", "mhmap5": ""}


        if mhmap1.geometry.contains(converted_point).any():
                maha_flood_risk_list["mhmap1"] = "Yes"
        else:
                maha_flood_risk_list["mhmap1"] = "No"

        if mhmap2.geometry.contains(converted_point).any():
                maha_flood_risk_list["mhmap2"] = "Yes"
        else:
                maha_flood_risk_list["mhmap2"] = "No"

        if mhmap3.geometry.contains(converted_point).any():
                maha_flood_risk_list["mhmap3"] = "Yes"
        else:
                maha_flood_risk_list["mhmap3"] = "No"

        if mhmap4.geometry.contains(normal_point).any():
                maha_flood_risk_list["mhmap4"] = "Yes"
        else:
                maha_flood_risk_list["mhmap4"] = "No"

        if mhmap5.geometry.contains(normal_point).any():
                maha_flood_risk_list["mhmap5"] = "Yes"
        else:
                maha_flood_risk_list["mhmap5"] = "No"



        yes_count = 0
        no_count = 0

        for key, value in maha_flood_risk_list.items():
            if key in ['mhmap1', 'mhmap2', 'mhmap3'] and value == 'Yes':
                yes_count += 1
            elif value == 'No':
                no_count += 1

        output = ""

        if maha_flood_risk_list['mhmap4'] == 'Yes' and maha_flood_risk_list['mhmap5'] == 'Yes':
            output = "high"
        elif (maha_flood_risk_list['mhmap4'] == 'Yes' or maha_flood_risk_list['mhmap5'] == 'Yes') and (yes_count == 3 or yes_count == 2):
            output = "high"
        elif (maha_flood_risk_list['mhmap4'] == 'Yes' or maha_flood_risk_list['mhmap5'] == 'Yes') and (yes_count == 1 or yes_count == 0):
            output = "moderate"
        elif yes_count == 3 or yes_count == 2:
            output = "moderate"
        elif yes_count == 1:
            output = "No"
        else :
            output = "No"
        print("----- maha------")
        print(output)

    elif season == "yala" or season == "යල":

        yala_flood_risk_list = {"ylmap1": "", "ylmap2": "", "ylmap3": "", "ylmap4": "", "ylmap5": "", "ylmap6": "", "ylmap7": ""}


        for key in yala_flood_risk_list:
            if key == "ylmap7":
                if ylmap7.geometry.contains(normal_point).any():
                    yala_flood_risk_list[key] = "Yes"
                else:
                    yala_flood_risk_list[key] = "No"
            else:
                if eval(key).geometry.contains(converted_point).any():
                    yala_flood_risk_list[key] = "Yes"
                else:
                    yala_flood_risk_list[key] = "No"

        yes_count = sum(value == "Yes" for value in yala_flood_risk_list.values())

        if yes_count >= 5:
            output = "High"
        elif yes_count >= 2:
            output = "Moderate"
        else:
            output = "No"

        # print("----- yala ------")
        # print(output)




    return {"flood_risk": f"{output}"}



class ADMBoundry(BaseModel):
    lat: float
    lon: float

@app.post("/ADMBoundry")
def ADMBoundry(request: ADMBoundry):

    lat = request.lat
    lon = request.lon
    #point = Point(lon, lat)

    # Create a Shapely Point geometry for the location
    location = Point(lon, lat)

    # Perform spatial query to find the province(s) containing the location
    intersected_provinces = provinces[provinces.geometry.contains(location)]

    District = intersected_provinces['ADM2_EN'].values
    GND = intersected_provinces['ADM4_EN'].values
    print(GND[0])

    return {"GND": f"{GND[0]}","District": f"{District[0]}"}



ngrok.set_auth_token('2Rw7whfUQ0dUtZhspiApkCSUqPk_7secQD2ho21Jfme31HXJo')

ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)

Public URL: https://5a7f-35-226-162-215.ngrok-free.app


INFO:     Started server process [405]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     116.206.246.246:0 - "POST /flood_risk HTTP/1.1" 200 OK


/usr/local/lib/python3.10/dist-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/usr/local/lib/python3.10/dist-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
<ipython-input-13-529aafb3d8bd>:43: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  x, y = transform(epsg_4326, epsg_5235, longitude, 

Punnalaikadduvan North
INFO:     116.206.246.246:0 - "POST /ADMBoundry HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [405]


# Run only the district and grama niladhari finding

In [ ]:
app1 = FastAPI()

origins = ["*"]

app1.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

class ADMBoundry(BaseModel):
    lat: float
    lon: float

@app1.post("/ADMBoundry")
def ADMBoundry(request: ADMBoundry):

    lat = request.lat
    lon = request.lon
    #point = Point(lon, lat)

    # Create a Shapely Point geometry for the location
    location = Point(lon, lat)

    # Perform spatial query to find the province(s) containing the location
    intersected_provinces = provinces[provinces.geometry.contains(location)]

    District = intersected_provinces['ADM2_EN']
    GND = intersected_provinces['ADM4_EN']

    return {"GND": f"{GND.values}","District": f"{District.values}"}


ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app1, port=8000)

Public URL: https://ecc3-35-226-162-215.ngrok.io


INFO:     Started server process [405]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [405]


# Run only the Flood risk

In [ ]:
app = FastAPI()

origins = ["*"]

app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)


from fastapi import FastAPI
from pydantic import BaseModel

app = FastAPI()

class GreetRequest(BaseModel):
    season: str
    lat: float
    lon: float

@app.get("/flood_risk")
def greet(request: GreetRequest):
    season = request.season
    #location = request.location

    #location = (11.228193200490075, 78.23941720427615)
    #season = "maha"
    latitude = request.lat
    longitude = request.lon

    # Define the coordinates in EPSG:4326 (latitude, longitude)
    # latitude = 9.761907869860229
    # longitude = 80.08335575093017

    # Define the EPSG codes for EPSG:4326 (WGS84) and EPSG:5235
    epsg_4326 = Proj(init='epsg:4326')
    epsg_5235 = Proj(init='epsg:5235')

    # Convert the coordinates to EPSG:5235
    x, y = transform(epsg_4326, epsg_5235, longitude, latitude)

    newlat = x
    newlon = y

    # Print the converted coordinates
    # print("-----------------------")
    # print(newlat,newlon)

    # for epsg:4326 point
    normal_point = geometry.Point(longitude,latitude)

    # for epsg:5235 point
    converted_point = geometry.Point(newlat,newlon)

    if season == "maha":

        maha_flood_risk_list = {"mhmap1": "", "mhmap2": "", "mhmap3": "", "mhmap4": "", "mhmap5": ""}


        if mhmap1.geometry.contains(converted_point).any():
                maha_flood_risk_list["mhmap1"] = "Yes"
        else:
                maha_flood_risk_list["mhmap1"] = "No"

        if mhmap2.geometry.contains(converted_point).any():
                maha_flood_risk_list["mhmap2"] = "Yes"
        else:
                maha_flood_risk_list["mhmap2"] = "No"

        if mhmap3.geometry.contains(converted_point).any():
                maha_flood_risk_list["mhmap3"] = "Yes"
        else:
                maha_flood_risk_list["mhmap3"] = "No"

        if mhmap4.geometry.contains(normal_point).any():
                maha_flood_risk_list["mhmap4"] = "Yes"
        else:
                maha_flood_risk_list["mhmap4"] = "No"

        if mhmap5.geometry.contains(normal_point).any():
                maha_flood_risk_list["mhmap5"] = "Yes"
        else:
                maha_flood_risk_list["mhmap5"] = "No"



        yes_count = 0
        no_count = 0

        for key, value in maha_flood_risk_list.items():
            if key in ['mhmap1', 'mhmap2', 'mhmap3'] and value == 'Yes':
                yes_count += 1
            elif value == 'No':
                no_count += 1

        output = ""

        if maha_flood_risk_list['mhmap4'] == 'Yes' and maha_flood_risk_list['mhmap5'] == 'Yes':
            output = "high"
        elif (maha_flood_risk_list['mhmap4'] == 'Yes' or maha_flood_risk_list['mhmap5'] == 'Yes') and (yes_count == 3 or yes_count == 2):
            output = "high"
        elif (maha_flood_risk_list['mhmap4'] == 'Yes' or maha_flood_risk_list['mhmap5'] == 'Yes') and (yes_count == 1 or yes_count == 0):
            output = "moderate"
        elif yes_count == 3 or yes_count == 2:
            output = "moderate"
        elif yes_count == 1:
            output = "No"
        else :
            output = "No"
        print("----- maha------")
        print(output)

    elif season == "yala":

        yala_flood_risk_list = {"ylmap1": "", "ylmap2": "", "ylmap3": "", "ylmap4": "", "ylmap5": "", "ylmap6": "", "ylmap7": ""}


        for key in yala_flood_risk_list:
            if key == "ylmap7":
                if ylmap7.geometry.contains(normal_point).any():
                    yala_flood_risk_list[key] = "Yes"
                else:
                    yala_flood_risk_list[key] = "No"
            else:
                if eval(key).geometry.contains(converted_point).any():
                    yala_flood_risk_list[key] = "Yes"
                else:
                    yala_flood_risk_list[key] = "No"

        yes_count = sum(value == "Yes" for value in yala_flood_risk_list.values())

        if yes_count >= 5:
            output = "High"
        elif yes_count >= 2:
            output = "Moderate"
        else:
            output = "No"

        # print("----- yala ------")
        # print(output)




    return {"risk": f"{output}"}


ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)

Public URL: https://e96f-35-226-162-215.ngrok.io


INFO:     Started server process [405]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [405]


# old parts

In [ ]:
location = (11.228193200490075, 78.23941720427615)
season = "maha"
latitude = location[0]
longitude = location[1]

# Define the coordinates in EPSG:4326 (latitude, longitude)
# latitude = 9.761907869860229
# longitude = 80.08335575093017

# Define the EPSG codes for EPSG:4326 (WGS84) and EPSG:5235
epsg_4326 = Proj(init='epsg:4326')
epsg_5235 = Proj(init='epsg:5235')

# Convert the coordinates to EPSG:5235
x, y = transform(epsg_4326, epsg_5235, longitude, latitude)

newlat = x
newlon = y

# Print the converted coordinates
# print("-----------------------")
# print(newlat,newlon)

# for epsg:4326 point
normal_point = geometry.Point(longitude,latitude)

# for epsg:5235 point
converted_point = geometry.Point(newlat,newlon)

/usr/local/lib/python3.10/dist-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/usr/local/lib/python3.10/dist-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
<ipython-input-8-e10e0f4f985e>:15: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  x, y = transform(epsg_4326, epsg_5235, longitude, l

In [ ]:
if season == "maha":

    maha_flood_risk_list = {"mhmap1": "", "mhmap2": "", "mhmap3": "", "mhmap4": "", "mhmap5": ""}


    if mhmap1.geometry.contains(converted_point).any():
            maha_flood_risk_list["mhmap1"] = "Yes"
    else:
            maha_flood_risk_list["mhmap1"] = "No"

    if mhmap2.geometry.contains(converted_point).any():
            maha_flood_risk_list["mhmap2"] = "Yes"
    else:
            maha_flood_risk_list["mhmap2"] = "No"

    if mhmap3.geometry.contains(converted_point).any():
            maha_flood_risk_list["mhmap3"] = "Yes"
    else:
            maha_flood_risk_list["mhmap3"] = "No"

    if mhmap4.geometry.contains(normal_point).any():
            maha_flood_risk_list["mhmap4"] = "Yes"
    else:
            maha_flood_risk_list["mhmap4"] = "No"

    if mhmap5.geometry.contains(normal_point).any():
            maha_flood_risk_list["mhmap5"] = "Yes"
    else:
            maha_flood_risk_list["mhmap5"] = "No"



    yes_count = 0
    no_count = 0

    for key, value in maha_flood_risk_list.items():
        if key in ['mhmap1', 'mhmap2', 'mhmap3'] and value == 'Yes':
            yes_count += 1
        elif value == 'No':
            no_count += 1

    output = ""

    if maha_flood_risk_list['mhmap4'] == 'Yes' and maha_flood_risk_list['mhmap5'] == 'Yes':
        output = "high"
    elif (maha_flood_risk_list['mhmap4'] == 'Yes' or maha_flood_risk_list['mhmap5'] == 'Yes') and (yes_count == 3 or yes_count == 2):
        output = "high"
    elif (maha_flood_risk_list['mhmap4'] == 'Yes' or maha_flood_risk_list['mhmap5'] == 'Yes') and (yes_count == 1 or yes_count == 0):
        output = "moderate"
    elif yes_count == 3 or yes_count == 2:
        output = "moderate"
    elif yes_count == 1:
        output = "No"
    else :
        output = "No"
    print("----- maha------")
    print(output)

elif season == "yala":

    yala_flood_risk_list = {"ylmap1": "", "ylmap2": "", "ylmap3": "", "ylmap4": "", "ylmap5": "", "ylmap6": "", "ylmap7": ""}


    for key in yala_flood_risk_list:
        if key == "ylmap7":
            if ylmap7.geometry.contains(normal_point).any():
                yala_flood_risk_list[key] = "Yes"
            else:
                yala_flood_risk_list[key] = "No"
        else:
            if eval(key).geometry.contains(converted_point).any():
                yala_flood_risk_list[key] = "Yes"
            else:
                yala_flood_risk_list[key] = "No"

    yes_count = sum(value == "Yes" for value in yala_flood_risk_list.values())

    if yes_count >= 5:
        output = "High"
    elif yes_count >= 2:
        output = "Moderate"
    else:
        output = "No"

    print("----- yala ------")
    print(output)



----- maha------
No


In [ ]:
!pip install geopandas
import geopandas as gpd
from shapely.geometry import Point

shapefile_path = "/content/drive/MyDrive/fyp/L4.zip"

app = FastAPI()

class ADMBoundry(BaseModel):
    lat: float
    lon: float

@app.get("/ADMBoundry")
def ADMBoundry(request: ADMBoundry):

    lat = request.lat
    lon = request.lon
    point = Point(lon, lat)

    #shapefile_path = "/content/drive/MyDrive/fyp/L4.zip"
    provinces = gpd.read_file(shapefile_path)

    # Create a Shapely Point geometry for the location
    location = Point(lon, lat)

    # Perform spatial query to find the province(s) containing the location
    intersected_provinces = provinces[provinces.geometry.contains(location)]

    GND = intersected_provinces['ADM4_EN']

    return {"message": f"{GND}"}


#print(intersected_provinces['ADM4_EN'],intersected_provinces['ADM3_EN'],intersected_provinces['ADM2_EN'],intersected_provinces['ADM1_EN'])

ERROR: Operation cancelled by user
